In [1]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token =  tokenizer.eos_token

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = model.to(device)

In [5]:
def output_hook(module, input, output):
    print(f'{module} : output')
    print(output.shape)
    print(output)
def input_hook(module, input, output):
    print(f'{module} : input')
    print(input)

def naive_noise_hook(module,input,output): #a rajouter : le fait de selectionner uniquement les bon token, et la bonne variance pour le bruit
    noise = torch.randn_like(output)#*sqrt(3*variance)
    return output+noise

In [7]:
# hook pour observer ce qu'il se passe dedans
hook1 = model.transformer.wte.register_forward_hook(input_hook)
hook2 = model.transformer.wte.register_forward_hook(output_hook)
hook3 = model.transformer.wpe.register_forward_hook(input_hook)
hook4 = model.transformer.wpe.register_forward_hook(output_hook)

In [8]:
#hook pour ajouter du bruit
noise_hook1 = model.transformer.drop.register_forward_hook(naive_noise_hook)

In [9]:
url = 'https://rome.baulab.info/data/dsets/known_1000.json'
response = requests.get(url) 
data = response.json()

In [48]:
prompts = [dict['prompt'] for dict in data][:100]
subjects = [dict['subject'] for dict in data][:100]
input= tokenizer(prompts, return_tensors="pt", padding= True, return_offsets_mapping= True)#return_offsets_mapping pour retenir ou sont les tokens
input = input.to(device)

In [49]:
mask = []
for j, prompt in enumerate(prompts):
    map = torch.zeros_like(input.input_ids[j], dtype=torch.int)#input_ids = id du token
    for i,t in enumerate(input.offset_mapping[j]):#offset_mapping = où est-ce qu'on a mis le padding, i = position, 
        
        if (prompts[j].find(subjects[j])-1<=t[0]) and (t[1]<=prompts[j].find(subjects[j])+len(subjects[j])):#sélectionne aussi le padding, qu'on élimine avec logical
            map[i] = 1
    mask.append(map)
masks_tensor = torch.stack(mask)
masks_tensor = torch.logical_and(masks_tensor, input.attention_mask).int()

In [50]:
input.offset_mapping.shape #n prompt,le plus gros prompt fait 27 tokens, (position de début du token, position de fin du token)

torch.Size([100, 21, 2])

In [51]:
def noise_hook(module,input,output):
    noise = torch.randn_like(output)*5
    noisy_output = output + noise * masks_tensor.unsqueeze(-1).float()
    print(output.shape, masks_tensor.shape)

    return noisy_output

In [52]:
# fonction qui sert à obtenir le logits du dernier non-padding token
def last_non_padding_token_logits(logits, attention_mask):
    # For each input, find the last non-padding token
    last_non_padding_logits = []
    
    for i in range(logits.size(0)):  # Loop over each prompt in the batch
        # Find the last non-padding token position
        non_padding_positions = (attention_mask[i] == 1).nonzero(as_tuple=True)[0]
        last_non_padding_token_index = non_padding_positions[-1]
        
        # Get the logits of the last non-padding token
        last_non_padding_logits.append(logits[i, last_non_padding_token_index])
    last_non_padding_logits = torch.stack(last_non_padding_logits)
    return last_non_padding_logits

In [53]:
#sans le noise
input= tokenizer(prompts, return_tensors="pt", padding= True).to(device)
with torch.no_grad():
    outputs = model(**input, output_hidden_states = True, output_attentions =True)


Embedding(50257, 1280) : input
(tensor([[   53,  7899,  5674,  ..., 50256, 50256, 50256],
        [ 3856,  1381,  7849,  ..., 50256, 50256, 50256],
        [16353,   856,    13,  ..., 50256, 50256, 50256],
        ...,
        [33704,   377,   952,  ..., 50256, 50256, 50256],
        [   43,  1536,  3418,  ..., 50256, 50256, 50256],
        [  464,  3303,   973,  ..., 50256, 50256, 50256]], device='cuda:0'),)
Embedding(50257, 1280) : output
torch.Size([100, 21, 1280])
tensor([[[ 0.0692,  0.0807, -0.0036,  ..., -0.0399,  0.0708, -0.0381],
         [ 0.1937, -0.0404, -0.0029,  ...,  0.1063, -0.0274,  0.0145],
         [-0.0031, -0.0195, -0.0433,  ..., -0.0395,  0.0050, -0.0218],
         ...,
         [-0.0419,  0.0848, -0.0512,  ..., -0.0083, -0.0447, -0.0274],
         [-0.0419,  0.0848, -0.0512,  ..., -0.0083, -0.0447, -0.0274],
         [-0.0419,  0.0848, -0.0512,  ..., -0.0083, -0.0447, -0.0274]],

        [[-0.0570, -0.0148, -0.0467,  ..., -0.0278, -0.0360, -0.0349],
         [ 0.0

OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 14.53 GiB of which 34.88 MiB is free. Process 2407007 has 3.14 GiB memory in use. Process 2414406 has 11.35 GiB memory in use. Of the allocated memory 11.13 GiB is allocated by PyTorch, and 146.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)